# Projeto BIG DATA I

## Objetivo de projeto

Como queremos demostrar que de fato a solução proposta traz uma melhora, foi solicitado implementar uma análise comparativa de resultados usando a antiga abordagem (usando pandas) e usando a nova proposta de solução (pyspark). Para isso, tome em consideração o seguinte:

1. Escolha dois conjuntos de dados interessantes, sendo que um deles é pequeno (menos de 10.000 linhas) e o outro bem maior (acima de 1.000.000 linhas).

2. Aplique todas as etapas de ETL nos dois conjuntos de dados usando pandas y pyspark. As etapas incluem: (1) Extração dos dados, por exemplo de um csv, (2) Tratamento dos dados (limpeza, alteração de nomes de colunas, criação de mais tabelas, transformação nas colunas, etc.), e, (3) Carregamento dos dados (salvar a transformação feita sobre os dados).

3. Lembre que cada etapa tem que ser feita usando unicamente pandas ou pyspark.

4. Como o objetivo é fazer uma análise comparativa, tome em consideração o tempo que demora cada etapa, para depois facilitar as comparações.

Boa sorte e divirta-se!!

## Importando Bibliotecas

In [46]:
import pandas as pd
from pyspark.sql import SparkSession
import timeit

## Importando arquivo csv

In [47]:
# Iniciando a sessão do Spark
spark = SparkSession.builder.appName("Pandas vs PySpark").getOrCreate()

# Lendo os datasets csv usando pandas
df_Netflix_pandas = pd.read_csv("datasets/Netflix_movies.csv")
df_Fraud_pandas = pd.read_csv("datasets/Fraudulent_Transactions_Data.csv")

# Lendo os datasets csv usando PySpark
df_Netflix_spark = spark.read.csv("datasets/Netflix_movies.csv", header=True, inferSchema=True)
df_Fraud_spark = spark.read.csv("datasets/Fraudulent_Transactions_Data.csv", header=True, inferSchema=True)

## Comparar o tempo de execução de algumas operações comuns usando pandas e PySpark

In [6]:
# Função para calcular o tempo
def print_time_difference(pandas_time, spark_time):
    # Cálculo da diferença percentual
    percentual_diff = (pandas_time - spark_time) / pandas_time * 100

    print("Tempo de execução (Pandas): ", pandas_time)
    print("Tempo de execução (PySpark): ", spark_time)
    print("Diferença percentual: ", percentual_diff, "%")

    if pandas_time < spark_time:
        print("Pandas foi mais rápido.")
    else:
        print("PySpark foi mais rápido.")

In [48]:
# Operação de contar a quantidade de linhas do dataset Netflix usando pandas
start = timeit.default_timer()
df_Netflix_pandas_lines = df_Netflix_pandas.count()[0]
pandas_time = timeit.default_timer() - start

# Operação de contar a quantidade de linhas do dataset Netflix usando PySpark
start = timeit.default_timer()
df_Netflix_spark_lines = df_Netflix_spark.count()
spark_time = timeit.default_timer() - start

print("Quantidade de linhas (Pandas): ", df_Netflix_pandas_lines)
print("Quantidade de linhas (PySpark): ", df_Netflix_spark_lines)
print_time_difference(pandas_time, spark_time)

Quantidade de linhas (Pandas):  3323
Quantidade de linhas (PySpark):  3323
Tempo de execução (Pandas):  0.007209400000192545
Tempo de execução (PySpark):  0.3617692000002535
Diferença percentual:  -4918.020917005459 %
Pandas foi mais rápido.


In [49]:
# Operação de contar a quantidade de linhas do dataset Netflix usando pandas
start = timeit.default_timer()
df_Fraud_pandas_lines = df_Fraud_pandas.count()[0]
pandas_time = timeit.default_timer() - start

# Operação de contar a quantidade de linhas do dataset Netflix usando PySpark
start = timeit.default_timer()
df_Fraud_spark_lines = df_Fraud_spark.count()
spark_time = timeit.default_timer() - start

print("Quantidade de linhas (Pandas): ", df_Fraud_pandas_lines)
print("Quantidade de linhas (PySpark): ", df_Fraud_spark_lines)
print_time_difference(pandas_time, spark_time)

Quantidade de linhas (Pandas):  6362620
Quantidade de linhas (PySpark):  6362620
Tempo de execução (Pandas):  0.7520202000000609
Tempo de execução (PySpark):  0.6993921999996928
Diferença percentual:  6.998216271366618 %
PySpark foi mais rápido.


In [50]:

# Contar Distintos - Netflix

# Descoberta de uma Coluna com um valor de distintos alto
print(list(df_Netflix_pandas.columns))

# Realizar distintos na coluna Actors
start = timeit.default_timer()
df_Netflix_pandas_lines = df_Netflix_pandas["actors"].unique()
pandas_time = timeit.default_timer() - start

# Operação de contar a quantidade de distintos do dataset Netflix usando PySpark

# -------------

print_time_difference(pandas_time, spark_time)

['Unnamed: 0', 'movie_name', 'Duration', 'year', 'genre', 'director', 'actors', 'country', 'rating', 'enter_in_netflix']
Tempo de execução (Pandas):  0.0014642999999523454
Tempo de execução (PySpark):  0.6993921999996928
Diferença percentual:  -47662.903778081956 %
Pandas foi mais rápido.


In [51]:

# Contar Distintos - Fraude

# Descoberta de uma Coluna com um valor de distintos alto
print(list(df_Fraud_pandas.columns))

# Realizar distintos na coluna "nameOrig"
start = timeit.default_timer()
df_Fraud_pandas_lines = df_Fraud_pandas["nameOrig"].unique()
pandas_time = timeit.default_timer() - start

# Operação de contar a quantidade de distintos do dataset Netflix usando PySpark

start = timeit.default_timer()
df_Fraud_spark_lines = df_Fraud_spark.select("nameOrig").distinct()
spark_time = timeit.default_timer() - start

print_time_difference(pandas_time, spark_time)

['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']
Tempo de execução (Pandas):  2.1959633999999824
Tempo de execução (PySpark):  0.006561199999850942
Diferença percentual:  99.7012154209924 %
PySpark foi mais rápido.


In [56]:
# Mudança do nome das Colunas - Pandas

new_columns = ["passo", "tipo", "quantidade", 
                "nome_origem", "balanço_antigo_origem", "balanço_novo_origem", 
                "nome_destino", "balanço_antigo_destino", "balanço_novo_destino", 
                "e_fraude", "flag_de_fraude"]

print("Colunas Antigas: " + str(list(df_Fraud_pandas.columns)))
df_Fraud_pandas.columns = new_columns
print("Novas colunas: " + str(list(df_Fraud_pandas.columns)))

# Mudança do nome das Colunas - Spark

print("\nColunas Antigas: " + str(list(df_Fraud_spark.columns)))
df_Fraud_spark = df_Fraud_spark.toDF(*new_columns)
print("Novas colunas: " + str(list(df_Fraud_spark.columns)))

Colunas Antigas: ['passo', 'tipo', 'quantidade', 'nome_origem', 'balanço_antigo_origem', 'balanço_novo_origem', 'nome_destino', 'balanço_antigo_destino', 'balanço_novo_destino', 'e_fraude', 'flag_de_fraude']
Novas colunas: ['passo', 'tipo', 'quantidade', 'nome_origem', 'balanço_antigo_origem', 'balanço_novo_origem', 'nome_destino', 'balanço_antigo_destino', 'balanço_novo_destino', 'e_fraude', 'flag_de_fraude']

Colunas Antigas: ['step', 'type', 'amount', 'nameOrig', 'oldbalanceOrg', 'newbalanceOrig', 'nameDest', 'oldbalanceDest', 'newbalanceDest', 'isFraud', 'isFlaggedFraud']
Novas colunas: ['passo', 'tipo', 'quantidade', 'nome_origem', 'balanço_antigo_origem', 'balanço_novo_origem', 'nome_destino', 'balanço_antigo_destino', 'balanço_novo_destino', 'e_fraude', 'flag_de_fraude']
